In [1]:
import cudaq

In [9]:
import cudaq

# Set the target to our density matrix simulator.
cudaq.set_target('density-matrix-cpu')
# cudaq.set_target('nvidia-mgpu')
# cudaq.set_target('nvidia')
print(cudaq.get_target())

# We will begin by defining an empty noise model that we will add
# these decoherence channels to.
noise = cudaq.NoiseModel()

# Bit flip channel with `1.0` probability of the qubit flipping 180 degrees.
bit_flip = cudaq.BitFlipChannel(.5)
# We will apply this channel to any X gate on the qubit, giving each X-gate
# a probability of `1.0` of undergoing an extra X-gate.
noise.add_channel('x', [0], bit_flip)

# construct a circuit
kernel = cudaq.make_kernel()
qubit = kernel.qalloc()

# Apply an X-gate to the qubit.
# It will remain in the |1> state with a probability of `1 - p = 0.9`.
kernel.x(qubit)
kernel.mz(qubit)

# noisy simulation

noisy_result = cudaq.sample(kernel, noise_model=noise, shots_count=int(1e2))
noisy_result.dump()
print(noisy_result)

# noiseless simulation

noiseless_result = cudaq.sample(kernel, shots_count = int(1e2))
noiseless_result.dump()
print(noiseless_result)

Target density-matrix-cpu
	simulator=dm
	platform=default
	description=The Density Matrix CPU Target provides a simulated QPU via OpenMP-enabled, CPU-only density matrix emulation.

{ 0:49 1:51 }

{ 1:100 }



In [15]:
import cudaq

def run_simulation(target, noise_model=False, shots_count=1500):
    # Set the target for the simulator
    cudaq.set_target(target)

    # Define the noise model if any
    noise = cudaq.NoiseModel()
    bit_flip = cudaq.BitFlipChannel(.5)  # 50% probability of a bit flip
    noise.add_channel('x', [0], bit_flip)

    # Construct a quantum circuit
    kernel = cudaq.make_kernel()
    qubit = kernel.qalloc(1)

    # Measure the qubit in the zbasis
    kernel.x(qubit)
    kernel.mz(qubit)

    # Perform the simulation
    noise_model = noise if noise_model == True else None
    result = cudaq.sample(kernel, noise_model=noise_model, shots_count=shots_count)
    result.dump()
    return result

# Test scenario
def test_simulation():
    # Run with CPU target (noisy)
    print("Noisy simulation (CPU):")
    noisy_result_cpu = run_simulation('density-matrix-cpu', noise_model=True)

    # Run with NVIDIA target (noisy)
    print("Noisy simulation (NVIDIA):")
    noisy_result_gpu = run_simulation('nvidia', noise_model=True)

    # Run with CPU target (noiseless)
    print("Noiseless simulation (CPU):")
    noiseless_result_cpu = run_simulation('density-matrix-cpu', noise_model=False)


    print("\nComparing results:")
    print(f"CPU noisy: {noisy_result_cpu}")
    print(f"GPU noisy: {noisy_result_gpu}")
    print(f"CPU noiseless: {noiseless_result_cpu}")


test_simulation()


Noisy simulation (CPU):
Noisy simulation (NVIDIA):
Noiseless simulation (CPU):

Comparing results:
CPU noisy: { 0:772 1:728 }

GPU noisy: { 1:1500 }

CPU noiseless: { 1:1500 }

